In [1]:
!pip install bitsandbytes
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 9.7 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM , BitsAndBytesConfig,TrainingArguments
import bitsandbytes as bnb
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model ,prepare_model_for_kbit_training
from torch.utils.data import DataLoader,Dataset
from torch.optim import AdamW
import torch
from tqdm import tqdm
from transformers import get_linear_schedule_with_warmup
from datasets import load_dataset
import numpy as np
import os
from huggingface_hub import login

os.environ['HF_TOKEN']="HF_TOKEN_HERE"
login(token=os.environ["HF_TOKEN"])

2026-02-18 17:32:02.825072: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771435923.020923      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771435923.071836      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771435923.521932      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771435923.521968      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771435923.521971      24 computation_placer.cc:177] computation placer alr

# Dataset

In [3]:
cp="neil-code/dialogsum-test"
ds=load_dataset(cp)

README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1999 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/499 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/499 [00:00<?, ? examples/s]

In [4]:
def add_instruction(dct):
    return {"dialouge_with_sum": f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
    
### Instruct: Summarise the following Conversation.

{dct['dialogue']}

### Output:
{dct['summary']} <|endoftext|>"""}

In [5]:
ds=ds.map(add_instruction)

Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

Map:   0%|          | 0/499 [00:00<?, ? examples/s]

Map:   0%|          | 0/499 [00:00<?, ? examples/s]

In [6]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'dialouge_with_sum'],
        num_rows: 1999
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'dialouge_with_sum'],
        num_rows: 499
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'dialouge_with_sum'],
        num_rows: 499
    })
})

In [7]:
ds=ds.remove_columns(['dialogue','id','topic','summary'])

In [8]:
train_dataset=ds['train']
val_dataset=ds['validation']

In [9]:
train_dataset

Dataset({
    features: ['dialouge_with_sum'],
    num_rows: 1999
})

In [10]:
print(ds['train'][0]['dialouge_with_sum'])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
    
### Instruct: Summarise the following Conversation.

#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#P

In [11]:
device='cuda' if torch.cuda.is_available() else "cpu"

In [12]:
cp="microsoft/phi-2"
tokenizer=AutoTokenizer.from_pretrained(cp)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [13]:
tokenizer.eos_token

'<|endoftext|>'

In [14]:
tokenizer.pad_token

In [15]:
tokenizer.pad_token=tokenizer.eos_token

In [16]:
# import torch.nn as nn
# import numpy as np
# def generate_preplexity_for_batch(logits,labels):
#   logits = torch.from_numpy(logits)
#   labels = torch.from_numpy(labels)
#   shift_logits = logits[:, :-1, :]
#   shift_labels = labels[:, 1:]
#   shift_logits = shift_logits.reshape(-1,shift_logits.shape[-1])
#   shift_labels = shift_labels.reshape(-1)
#   criterion = nn.CrossEntropyLoss()
#   ppl=criterion(shift_logits,shift_labels)
#   return np.exp(ppl)

In [17]:
# def compute_metrics(eval_preds):
#     logits, labels = eval_preds
#     ppl = generate_preplexity_for_batch(logits,labels)
#     return {'perplexity':ppl}

In [18]:
# args=TrainingArguments('/kaggle/working/llama',per_device_eval_batch_size=1,per_device_train_batch_size=1,num_train_epochs=1,push_to_hub=True,report_to="none",logging_steps=100,gradient_accumulation_steps=8)
# trainer=SFTTrainer(args=args,model=model,processing_class=tokenizer,train_dataset=train_dataset,eval_dataset=test_dataset,compute_metrics=compute_metrics)

In [19]:
# try:
#     model.train()
#     # trainer.train()
#     model.eval()
#     trainer.evaluate()
# except Exception as e:
#     torch.cuda.empty_cache()
#     print(e)

In [20]:
train_dataset[0]

{'dialouge_with_sum': "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n    \n### Instruct: Summarise the following Conversation.\n\n#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I j

In [21]:
def customCollate(input_dct):
    texts=[i['dialouge_with_sum'] for i in input_dct]
    dct=tokenizer(texts,padding=True,truncation=True,return_tensors='pt')
    labels=dct['input_ids'].clone()
    labels[labels==tokenizer.pad_token_id]=-100
    dct={k:v.to(device) for k,v in dct.items()}
    labels=labels.to(device)
    return {"texts":dct,"labels":labels}

In [22]:
batch_size=1
train_ds=DataLoader(train_dataset,batch_size=batch_size,collate_fn=customCollate)
val_ds=DataLoader(val_dataset,batch_size=batch_size,collate_fn=customCollate)

In [23]:
next(iter(train_ds))

{'texts': {'input_ids': tensor([[21106,   318,   281, 12064,   326,  8477,   257,  4876,    11, 20312,
             351,   281,  5128,   326,  3769,  2252,  4732,    13, 19430,   257,
            2882,   326, 20431, 32543,   262,  2581,    13,   198, 50284,   198,
           21017, 20689,    25,  5060,  3876,   786,   262,  1708, 42427,    13,
             198,   198,     2, 15439,    16,     2,    25, 15902,    11,  1770,
              13,  4176,    13,   314,  1101,  9356, 32534,    13,  4162,   389,
             345,   994,  1909,    30,   198,     2, 15439,    17,     2,    25,
             314,  1043,   340,   561,   307,   257,   922,  2126,   284,   651,
             257,  2198,    12,   929,    13,   198,     2, 15439,    16,     2,
              25,  3363,    11,   880,    11,   345,  4398,   470,   550,   530,
             329,   642,   812,    13,   921,   815,   423,   530,   790,   614,
              13,   198,     2, 15439,    17,     2,    25,   314,   760,    13,
      

# Model

In [24]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)
model=AutoModelForCausalLM.from_pretrained(cp,quantization_config=quantization_config,device_map="auto")

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [25]:
model=prepare_model_for_kbit_training(model)

In [26]:
model.gradient_checkpointing_enable()

In [27]:
lora_config=LoraConfig(r=32,lora_alpha=32,target_modules=["q_proj", "v_proj",'o_proj','dense'],lora_dropout=0.05,bias="none",task_type="CAUSAL_LM")
model=get_peft_model(model,peft_config=lora_config)

In [28]:
total_trainable_params=0
total_params=0
for params in model.parameters():
    total_params+=params.numel()
    if params.requires_grad:
        total_trainable_params+=params.numel()
total_trainable_params*100/total_params

1.0232530252915373

In [29]:
def train_epoch(model,train_ds,optimizer,scheduler,accumulation_step=4):
    tot_loss=0
    step_comp=0
    i=0
    progress = tqdm(train_ds, desc="Training")
    for batch in progress:
        try:
            i+=1
            out=model(**batch['texts'],labels=batch['labels'])
            loss=out.loss
            loss.backward()
    
            tot_loss+=loss.item() 
            progress.set_postfix({'loss': f'{loss.item():.4f}'})
            if (i)%accumulation_step==0 or i == len(train_ds)-1:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
        except Exception as E:
            print(f"Error at batch {i}: {E}")
            torch.cuda.empty_cache()
            optimizer.zero_grad()

    
    return tot_loss/len(train_ds)

In [30]:
def val(model, val_ds):
    progress = tqdm(val_ds, desc="Validating")
    total_loss = 0
    num_batches = 0
    
    for batch in progress:
        try:
            with torch.no_grad():
                out = model(**batch['texts'], labels=batch['labels'])
                if not np.isnan(out.loss.item()):
                    total_loss += out.loss.item()
                    num_batches += 1
                progress.set_postfix({'loss': f'{out.loss.item():.4f}'})
        except Exception as e:
            print(f"Error processing batch: {e}")
            torch.cuda.empty_cache()
            continue
    
    avg_loss = total_loss / num_batches if num_batches > 0 else float('inf')
    return avg_loss

In [31]:
optimizer=AdamW([i for i in model.parameters() if i.requires_grad],lr=2e-4)

n_epochs=2
total_steps = len(train_ds) * n_epochs
warmup_steps=int(0.1 * total_steps)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

In [32]:
%%time
# model output before training
batch1=next(iter(train_ds))
input1=tokenizer.batch_decode(batch1['texts']['input_ids'])[0].split("### Output")[0]+"### Output:\n"
input1=tokenizer(input1,return_tensors='pt')
input1={k:v.to(device) for k,v in input1.items()}
out=model.generate(**input1)
print(tokenizer.batch_decode(out)[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
    
### Instruct: Summarise the following Conversation.

#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#P

In [33]:
lss=val(model,val_ds)

print(f"(perplexity before training :",f'{np.exp(lss):.4f}')

Validating: 100%|██████████| 499/499 [03:49<00:00,  2.18it/s, loss=2.1604]

(perplexity before training : 8.7491


In [34]:
for epoch in range(1,n_epochs+1):
    print(f"Epoch {epoch}")
    train_loss=train_epoch(model,train_ds,optimizer,scheduler)
    print(f"loss in epoch {epoch} :",f'{train_loss:.4f}, perplexity : {np.exp(train_loss)}')
    lss=val(model,val_ds)
    print(f"(val loss after epoch {epoch} :",f'{lss:.4f} perplexity : {np.exp(lss)}')


Epoch 1


Training: 100%|██████████| 1999/1999 [31:10<00:00,  1.07it/s, loss=1.2883]


loss in epoch 1 : 1.3831, perplexity : 3.9873492991687147


Validating: 100%|██████████| 499/499 [03:49<00:00,  2.17it/s, loss=1.4990]


(val loss after epoch 1 : 1.2660 perplexity : 3.546723159155149
Epoch 2


Training: 100%|██████████| 1999/1999 [31:10<00:00,  1.07it/s, loss=1.2420]


loss in epoch 2 : 1.2350, perplexity : 3.43844816613262


Validating: 100%|██████████| 499/499 [03:49<00:00,  2.17it/s, loss=1.4930]

(val loss after epoch 2 : 1.2636 perplexity : 3.5381209258976334


In [35]:
%%time
# model output after training
batch1=next(iter(train_ds))
input1=tokenizer.batch_decode(batch1['texts']['input_ids'])[0].split("### Output")[0]+"### Output:\n"
input1=tokenizer(input1,return_tensors='pt')
input1={k:v.to(device) for k,v in input1.items()}
out=model.generate(**input1)
print(tokenizer.batch_decode(out)[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
    
### Instruct: Summarise the following Conversation.

#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#P

In [36]:
!mkdir /kaggle/working/phi-2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [37]:
model.save_pretrained("/kaggle/working/phi-2")

In [38]:
from huggingface_hub import login, upload_folder
 
# (optional) Login with your Hugging Face credentials
login()

# Push your model files
upload_folder(folder_path="/kaggle/working/phi-2", repo_id="Pankaj121212/phi-2-nex", repo_type="model")

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/Pankaj121212/phi-2-nex/commit/64faf5b89db36918361883fa2deee7b4d5299e59', commit_message='Upload folder using huggingface_hub', commit_description='', oid='64faf5b89db36918361883fa2deee7b4d5299e59', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Pankaj121212/phi-2-nex', endpoint='https://huggingface.co', repo_type='model', repo_id='Pankaj121212/phi-2-nex'), pr_revision=None, pr_num=None)